In [1]:
import sys
import os

# Giả sử bạn đang ở: /path/to/projects/notebooks
# Ta thêm thư mục Text2CAD vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../Text2CAD'))
sys.path.append(project_root)


In [2]:
import json
import pandas as pd
import numpy as np
import torch
from CadSeqProc.cad_sequence import CADSequence
from CadSeqProc.utility.utils import normalize_pc, chamfer_dist
from CadSeqProc.utility.macro import CAD_CLASS_INFO, N_BIT
from CadSeqProc.utility.utils import (create_path_with_time,ensure_dir)
import os
from tqdm import tqdm
import json
import math
from collections import defaultdict
from datasets import load_dataset

In [2]:
# '''Read data from jsonl'''
# def read_data(path):
#     print(f"Reading data from {path}")

#     try:
#         with open(path, 'r', encoding='utf-8') as f:
#             jsonl_data = [json.loads(line) for line in f]
#     except FileNotFoundError:
#         print(f"Error: Input file not found at {path}")
#     return jsonl_data

In [5]:
# # path = 'predictions_qwen25_1-6150.jsonl'
# # path = 'predictions_llama.jsonl'
# path = '../llama/predictions_llama_en.jsonl'
# # path = '../llama/vi/predictions_llama_vi.jsonl'
# path = '../qwen/predictions_qwen_1.5B_Instruct_en_0-100.jsonl'
# data = read_data(path)

In [3]:
d = load_dataset("wanhin/test_reasoning_1_2e")


In [4]:
d

DatasetDict({
    no_sampling_en_0_1325: Dataset({
        features: ['uid', 'input', 'reasoning', 'predicted_output', 'ground_truth_output'],
        num_rows: 1325
    })
    no_sampling_en_1325_2650: Dataset({
        features: ['uid', 'input', 'reasoning', 'predicted_output', 'ground_truth_output'],
        num_rows: 1325
    })
    no_sampling_en_5300_6625: Dataset({
        features: ['uid', 'input', 'reasoning', 'predicted_output', 'ground_truth_output'],
        num_rows: 1325
    })
    no_sampling_en_3975_5300: Dataset({
        features: ['uid', 'input', 'reasoning', 'predicted_output', 'ground_truth_output'],
        num_rows: 1325
    })
    no_sampling_en_2650_3975: Dataset({
        features: ['uid', 'input', 'reasoning', 'predicted_output', 'ground_truth_output'],
        num_rows: 1325
    })
    no_sampling_en_6625_7924: Dataset({
        features: ['uid', 'input', 'reasoning', 'predicted_output', 'ground_truth_output'],
        num_rows: 1299
    })
    no_sampling_v

In [5]:
data_en = []
data_vi = []
for split in d:
    if split.startswith('no_sampling_en'):
        data_en.extend(d[split])
    else:
        data_vi.extend(d[split])
    # data.extend(d[split])    
print(len(data_en))
print(len(data_vi))

7924
3200


In [9]:
x = 0
data = []
for r in data_en:
    if r['predicted_output'] == '':
        x+=1
        continue
    try:
        h = json.loads(r['predicted_output'].replace("'", '"'))
    except:
        x += 1
        continue

    data.append(r)
x
    

185

In [5]:
# x = 0
# data = []
# for r in d['no_sampling_446600_448600']:
#     if r['predicted_output'] =='':
#         x+= 1
#     else:
#         data.append(r)
# x

72

In [7]:
# data_en = []
# data_vi = []
# for r in data:
#     if 'Suy' in r['reasoning']:
#         data_vi.append(r)
#     else:
#         data_en.append(r)
# print(len(data_en), '  -  ', len(data_vi))

950   -   978


In [17]:
data = [_ for _ in d["no_sampling_en"]]

In [10]:
len(data)

7739

In [23]:
# haizz = d['no_sampling_en']
# s = {}
# for sample in haizz:
#     s[sample['uid']] = sample['ground_truth_output']

In [24]:
# h = []
# for sample in data:
#     uid = list(sample.keys())[0]
#     val = list(sample.values())[0][0].split('### Response:')[1].replace('<|endoftext|>','')
#     new = {
#         'uid': uid,
#         'ground_truth_output': s[uid],
#         'predicted_output': val
#     }
#     h.append(new)

In [25]:
# data = h

In [14]:
error = 0
output_data = {} 
error_samples = []
n = []
level = 'expert'
error_gt = 0
for sample in tqdm(data, desc="Processing samples"):
    uid = sample['uid']
    
    if uid not in output_data:
        output_data[uid] = {}
        
    if level not in output_data[uid]:
        output_data[uid][level] = {}
        
    try:
        try:
            gt_cad = CADSequence.from_minimal_json(json.loads(sample['ground_truth_output'].replace("'",'"'))).create_cad_model().sample_points(n_points=8192)
        except:
            try:
                zzz = json.loads(sample['ground_truth_output'].replace("'",'"')[:-1])
                gt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
            except Exception as e:
                error_gt += 1
                continue

            
        pred_cad = CADSequence.from_minimal_json(json.loads(sample['predicted_output'].replace("'",'"'))).create_cad_model().sample_points(n_points=8192)
        # gt_cad.sample_points(n_points=8192)
        # pred_cad.sample_points(n_points=8192)

        cd = (
            chamfer_dist(
                normalize_pc(gt_cad.points),
                normalize_pc(pred_cad.points),
            )
            * 1000
        )
        is_invalid = False
    except Exception as e:
        cd = -1
        is_invalid = True
        error += 1
        error_samples.append(sample)
        n.append(e)
        pred_cad = None
    output_data[uid][level]['pred_cad_vec'] = [pred_cad] # Store as a list with one element
    output_data[uid][level]['gt_cad_vec'] = gt_cad      # Store GT vector
    output_data[uid][level]['cd'] = [cd]                # Store CD as a list with one element
    output_data[uid][level]['is_invalid'] = is_invalid  # Store the invalidity flag
len(error_samples)

Processing samples: 100%|██████████████████████████████████████████████████████████| 7739/7739 [19:27<00:00,  6.63it/s]


40

In [15]:
set(n)

{OSError('File not written to disk on path .temp_159392.stl'),
 OSError('File not written to disk on path .temp_426516.stl'),
 OSError('File not written to disk on path .temp_466882.stl'),
 OSError('File not written to disk on path .temp_515810.stl'),
 OSError('File not written to disk on path .temp_67992.stl'),
 OSError('File not written to disk on path .temp_978225.stl'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('Invalid Model'),
 Exception('In

In [4]:
x= {"final_shape": "", "parts": {"part_1": {"coordinate_system": {"Euler Angles": [0.0, 0.0, -90.0], "Translation Vector": [0.0004, 0.0171, 0.0]}, "sketch": {"face_1": {"loop_1": {"line_1": {"Start Point": [0.1738, 0.3831], "End Point": [0.1775, 0.4219]}, "arc_1": {"Start Point": [0.1775, 0.4219], "Mid Point": [0.0, 0.4192], "End Point": [0.1738, 0.3831]}}, "loop_2": {"circle_1": {"Center": [0.0889, 0.4107], "Radius": 0.0179}}}, "face_2": {"loop_1": {"line_1": {"Start Point": [0.0, 0.4018], "End Point": [0.0356, 0.0482]}, "arc_1": {"Start Point": [0.0356, 0.0482], "Mid Point": [0.1241, 0.0939], "End Point": [0.0908, 0.0]}, "line_2": {"Start Point": [0.0908, 0.0], "End Point": [0.3405, 0.009]}, "arc_2": {"Start Point": [0.3405, 0.009], "Mid Point": [0.2942, 0.0536], "End Point": [0.3405, 0.0982]}, "line_3": {"Start Point": [0.3405, 0.0982], "End Point": [0.1948, 0.1034]}, "arc_3": {"Start Point": [0.1948, 0.1034], "Mid Point": [0.1627, 0.1186], "End Point": [0.1518, 0.1523]}, "line_4": {"Start Point": [0.1518, 0.1523], "End Point": [0.1738, 0.3831]}, "arc_4": {"Start Point": [0.1738, 0.3831], "Mid Point": [0.0793, 0.3219], "End Point": [0.0, 0.4018]}}}, "face_3": {"loop_1": {"arc_1": {"Start Point": [0.0003, 0.4219], "Mid Point": [0.0889, 0.5], "End Point": [0.1775, 0.4219]}, "line_1": {"Start Point": [0.1775, 0.4219], "End Point": [0.142, 0.7031]}, "arc_2": {"Start Point": [0.142, 0.7031], "Mid Point": [0.0889, 0.6429], "End Point": [0.0357, 0.7031]}, "line_2": {"Start Point": [0.0357, 0.7031], "End Point": [0.0003, 0.4219]}}, "loop_2": {"circle_1": {"Center": [0.071, 0.6161], "Radius": 0.0179}}}, "face_4": {"loop_1": {"circle_1": {"Center": [0.0889, 0.0536], "Radius": 0.0536}}, "loop_2": {"circle_1": {"Center": [0.0889, 0.0536], "Radius": 0.0179}}}, "face_5": {"loop_1": {"circle_1": {"Center": [0.0889, 0.6964], "Radius": 0.0536}}, "loop_2": {"circle_1": {"Center": [0.0889, 0.6964], "Radius": 0.0179}}}, "face_6": {"loop_1": {"line_1": {"Start Point": [0.1738, 0.3831], "End Point": [0.1775, 0.4219]}, "arc_1": {"Start Point": [0.1775, 0.4219], "Mid Point": [0.1778, 0.4023], "End Point": [0.1738, 0.3831]}}}, "face_7": {"loop_1": {"circle_1": {"Center": [0.3389, 0.0536], "Radius": 0.0446}}, "loop_2": {"circle_1": {"Center": [0.3389, 0.0536], "Radius": 0.0179}}}}, "extrusion": {"extrude_depth_towards_normal": 0.0171, "extrude_depth_opposite_normal": 0.0, "sketch_scale": 0.75, "operation": "NewBodyFeatureOperation"}, "description": {"shape": "", "length": 0.75, "width": 0.0171, "height": 0.75}}}}


In [5]:
y= {"final_shape": "", "parts": {"part_1": {"coordinate_system": {"Euler Angles": [0.0, 0.0, -90.0], "Translation Vector": [0.0004, 0.0171, 0.0]}, "sketch": {"face_1": {"loop_1": {"line_1": {"Start Point": [0.1738, 0.3831], "End Point": [0.1775, 0.4219]}, "arc_1": {"Start Point": [0.1775, 0.4219], "Mid Point": [0.0, 0.4192], "End Point": [0.1738, 0.3831]}}, "loop_2": {"circle_1": {"Center": [0.0889, 0.4107], "Radius": 0.0179}}}, "face_2": {"loop_1": {"line_1": {"Start Point": [0.0, 0.4018], "End Point": [0.0356, 0.0482]}, "arc_1": {"Start Point": [0.0356, 0.0482], "Mid Point": [0.1241, 0.0939], "End Point": [0.0908, 0.0]}, "line_2": {"Start Point": [0.0908, 0.0], "End Point": [0.3405, 0.009]}, "arc_2": {"Start Point": [0.3405, 0.009], "Mid Point": [0.2942, 0.0536], "End Point": [0.3405, 0.0982]}, "line_3": {"Start Point": [0.3405, 0.0982], "End Point": [0.1948, 0.1034]}, "arc_3": {"Start Point": [0.1948, 0.1034], "Mid Point": [0.1627, 0.1186], "End Point": [0.1518, 0.1523]}, "line_4": {"Start Point": [0.1518, 0.1523], "End Point": [0.1738, 0.3831]}, "arc_4": {"Start Point": [0.1738, 0.3831], "Mid Point": [0.0793, 0.3219], "End Point": [0.0, 0.4018]}}}, "face_3": {"loop_1": {"arc_1": {"Start Point": [0.0003, 0.4219], "Mid Point": [0.0889, 0.5], "End Point": [0.1775, 0.4219]}, "line_1": {"Start Point": [0.1775, 0.4219], "End Point": [0.142, 0.7031]}, "arc_2": {"Start Point": [0.142, 0.7031], "Mid Point": [0.0889, 0.6429], "End Point": [0.0357, 0.7031]}, "line_2": {"Start Point": [0.0357, 0.7031], "End Point": [0.0003, 0.4219]}}, "loop_2": {"circle_1": {"Center": [0.071, 0.6161], "Radius": 0.0179}}}, "face_4": {"loop_1": {"circle_1": {"Center": [0.0889, 0.6964], "Radius": 0.0536}}, "loop_2": {"circle_1": {"Center": [0.0889, 0.6964], "Radius": 0.0179}}}, "face_5": {"loop_1": {"circle_1": {"Center": [0.0889, 0.0536], "Radius": 0.0179}}, "loop_2": {"arc_1": {"Start Point": [0.0889, 0.0], "Mid Point": [0.0869, 0.1071], "End Point": [0.0908, 0.0]}, "line_1": {"Start Point": [0.0908, 0.0], "End Point": [0.0889, 0.0]}}}, "face_6": {"loop_1": {"arc_1": {"Start Point": [0.0889, 0.0], "Mid Point": [0.0905, 0.0], "End Point": [0.0908, 0.0]}, "line_1": {"Start Point": [0.0908, 0.0], "End Point": [0.0889, 0.0]}}}, "face_7": {"loop_1": {"circle_1": {"Center": [0.3389, 0.0536], "Radius": 0.0446}}, "loop_2": {"circle_1": {"Center": [0.3389, 0.0536], "Radius": 0.0179}}}}, "extrusion": {"extrude_depth_towards_normal": 0.0171, "extrude_depth_opposite_normal": 0.0, "sketch_scale": 0.75, "operation": "NewBodyFeatureOperation"}, "description": {"shape": "", "length": 0.75, "width": 0.0171, "height": 0.75}}}}


In [13]:
# u = {"parts":{"part_1":{"coordinate_system":{"Euler Angles":[0.0,0.0,-90.0],"Translation Vector":[0.0,0.1432,0.0]},"sketch":{"face_1":{"loop_1":{"circle_1":{"Center":[0.0895,0.4099],"Radius":0.0895}},"loop_2":{"circle_1":{"Center":[0.0895,0.4099],"Radius":0.0179}}},"face_2":{"loop_1":{"line_1":{"Start Point":[0.0005,0.4005],"End Point":[0.0375,0.052]},"arc_1":{"Start Point":[0.0375,0.052],"Mid Point":[0.1252,0.0898],"End Point":[0.0925,0.0001]},"line_2":{"Start Point":[0.0925,0.0001],"End Point":[0.3414,0.0072]},"arc_2":{"Start Point":[0.3414,0.0072],"Mid Point":[0.2953,0.0513],"End Point":[0.3401,0.0967]},"line_3":{"Start Point":[0.3401,0.0967],"End Point":[0.2238,0.0967]},"arc_3":{"Start Point":[0.2238,0.0967],"Mid Point":[0.1921,0.1098],"End Point":[0.179,0.1415]},"line_4":{"Start Point":[0.179,0.1415],"End Point":[0.179,0.4099]},"arc_4":{"Start Point":[0.179,0.4099],"Mid Point":[0.0942,0.3206],"End Point":[0.0005,0.4005]}}},"face_3":{"loop_1":{"arc_1":{"Start Point":[0.0007,0.4213],"Mid Point":[0.0896,0.4994],"End Point":[0.1783,0.4211]},"line_1":{"Start Point":[0.1783,0.4211],"End Point":[0.1428,0.703]},"arc_2":{"Start Point":[0.1428,0.703],"Mid Point":[0.0929,0.6427],"End Point":[0.0358,0.6963]},"line_2":{"Start Point":[0.0358,0.6963],"End Point":[0.0007,0.4213]}},"loop_2":{"circle_1":{"Center":[0.0716,0.6158],"Radius":0.0179}}},"face_4":{"loop_1":{"circle_1":{"Center":[0.0895,0.6963],"Radius":0.0537}},"loop_2":{"circle_1":{"Center":[0.0895,0.6963],"Radius":0.0179}}},"face_5":{"loop_1":{"circle_1":{"Center":[0.0895,0.052],"Radius":0.0179}},"loop_2":{"arc_1":{"Start Point":[0.0895,0.0],"Mid Point":[0.088,0.1039],"End Point":[0.0925,0.0001]},"line_1":{"Start Point":[0.0925,0.0001],"End Point":[0.0895,0.0]}}},"face_6":{"loop_1":{"arc_1":{"Start Point":[0895,0.0],"Mid Point":[0.091,0.0],"End Point":[0.0925,0.0001]},"line_1":{"Start Point":[0.0925,0.0001],"End Point":[0.0895,0.0]}},"loop_2":{"circle_1":{"Center":[0.0895,0.052],"Radius":0.0179}}},"face_7":{"loop_1":{"circle_1":{"Center":[0.3401,0.052],"Radius":0.0447}},"loop_2":{"circle_1":{"Center":[0.3401,0.052],"Radius":0.0179}}}},"extrusion":{"extrude_depth_towards_normal":0.1432,"extrude_depth_opposite_normal":0.0,"sketch_scale":0.75,"operation":"NewBodyFeatureOperation"}}}}


In [43]:
### EMD 
import numpy as np
import ot # Đây là thư viện POT

def calculate_emd(pc1, pc2):
    """
    Tính toán Earth Mover's Distance (EMD) giữa hai đám mây điểm.

    Args:
        pc1 (np.ndarray): Đám mây điểm thứ nhất, shape (N, 3).
        pc2 (np.ndarray): Đám mây điểm thứ hai, shape (M, 3).
                          Lý tưởng nhất là N = M.

    Returns:
        float: Giá trị EMD (càng thấp càng tốt).
    """
    # EMD yêu cầu các phân bố (weights) cho mỗi điểm.
    # Với đám mây điểm đồng nhất, mỗi điểm có trọng số bằng nhau.
    n_points1 = pc1.shape[0]
    n_points2 = pc2.shape[0]
    
    # Phân bố trọng số cho đám mây điểm 1 (tổng bằng 1)
    a = np.ones(n_points1) / n_points1
    
    # Phân bố trọng số cho đám mây điểm 2 (tổng bằng 1)
    b = np.ones(n_points2) / n_points2

    # Tính toán ma trận chi phí (Cost Matrix).
    # M[i, j] là chi phí để "di chuyển" điểm i từ pc1 đến điểm j ở pc2.
    # Chi phí này thường là khoảng cách bình phương Euclid (squared Euclidean distance).
    # Thư viện POT có hàm ot.dist() để tính toán việc này một cách hiệu quả.
    M = ot.dist(pc1, pc2, metric='sqeuclidean')

    # Tính toán EMD. ot.emd2 trả về tổng chi phí vận chuyển.
    # Đây chính là giá trị EMD mà chúng ta cần.
    emd_value = ot.emd2(a, b, M)

    return emd_value

# --- VÍ DỤ SỬ DỤNG ---
if __name__ == '__main__':

    index = 1000
    gt = json.loads(data[index]['ground_truth_output'])
    pr = json.loads(data[100]['predicted_output'].replace("'", '"'))
    ground_truth_cad = CADSequence.from_minimal_json(gt).create_cad_model().sample_points(n_points = 8192, type= 'uniform')
    predicted_cad = CADSequence.from_minimal_json(gt).create_cad_model().sample_points(n_points = 8192, type= 'uniform')

    
    # Đám mây điểm ground-truth (ví dụ: một khối lập phương)
    ground_truth_pc = normalize_pc(ground_truth_cad.points)
    
    # Đám mây điểm dự đoán (ví dụ: một khối lập phương bị dịch đi một chút)
    predicted_pc = normalize_pc(predicted_cad.points)

    print(f"Shape của đám mây điểm ground-truth: {ground_truth_pc.shape}")
    print(f"Shape của đám mây điểm dự đoán: {predicted_pc.shape}")

    # Tính EMD
    emd_distance = calculate_emd(ground_truth_pc, predicted_pc)

    print(f"\nGiá trị Earth Mover's Distance (EMD) là: {emd_distance:.6f}")


Shape của đám mây điểm ground-truth: (8192, 3)
Shape của đám mây điểm dự đoán: (8192, 3)

Giá trị Earth Mover's Distance (EMD) là: 0.000750


In [41]:
sum(emd_result * 100 )/len(emd_result)

0.10943067841697889

In [42]:
sum(cd_result )/len(cd_result)

0.20224423093909216

In [ ]:
print(sum(emd_result2 * 100 )/len(emd_result))
print(sum(cd_result2)/len(cd_result))

In [ ]:
cd_result2 = []
emd_result2 = []
for row in tqdm(data[:10]):
    gt = json.loads(data[index]['ground_truth_output'])
    pt = json.loads(data[index]['predicted_output'])
    ground_truth_cad = CADSequence.from_minimal_json(gt).create_cad_model().sample_points(n_points = 8192, type= 'uniform')
    ground_truth_pc = normalize_pc(ground_truth_cad.points)
    predict_cad = CADSequence.from_minimal_json(pt).create_cad_model().sample_points(n_points = 8192, type= 'uniform')
    predict_pc = normalize_pc(predict_cad.points)
    emd = calculate_emd(ground_truth_pc, predict_pc)
    cd = (
            chamfer_dist(
                ground_truth_pc,
                predict_pc,
            )
            * 1000
        )
    
    cd_result2.append(cd)
    emd_result2.append(emd)

    

 70%|██████████████████████████████████████████████████████████                         | 7/10 [01:23<00:36, 12.32s/it]

In [25]:
random_pc = np.random.rand(8192, 3)
bad_emd_distance = calculate_emd(ground_truth_pc, random_pc)
print(f"Giá trị EMD cho một mô hình tệ là: {bad_emd_distance:.6f} (lớn hơn đáng kể)")

Giá trị EMD cho một mô hình tệ là: 0.282909 (lớn hơn đáng kể)


In [7]:
# output_data = {}
# g = CADSequence.from_minimal_json(x).create_cad_model().sample_points(n_points = 8192, type= 'uniform')
h = CADSequence.from_minimal_json(x).create_cad_model()
h = h.sample_points(n_points = 8192, type= 'uniform')
g = CADSequence.from_minimal_json(x).create_cad_model()
g = g.sample_points(n_points = 8192, type= 'uniform')
# g.sample_points()
# h.sample_points()
cd = (
            chamfer_dist(
                normalize_pc(h.points),
                normalize_pc(g.points),
            )
            * 1000
        )
cd
# output_data['1'] = {}
# output_data['1']['expert']= {}
# output_data['1']['expert']['pred_cad_vec'] = [h] # Store as a list with one element
# output_data['1']['expert']['gt_cad_vec'] = g      # Store GT vector
# output_data['1']['expert']['cd'] = [cd]                # Store CD as a list with one element
# output_data['1']['expert']['is_invalid'] = False  # Store the invalidity flag

0.03992338241797718

In [14]:
# h.save_stp('x1', 'cad')

In [15]:
# full = ""
# for sample in errors:
#     # full += f" ##Input: {sample['input']} \n ##output: {sample['predicted_output']} \n ##ground_truth: {sample['ground_truth_output']} \n ===================================="
    
#     if sample['uid'] == '0054/00541629':
#         print( '##INPUT: ', sample['input'])
    
#         print( '##GROUND_TRUTH: ',sample['ground_truth_output'])
#         print('##PREDICT: ',sample['predicted_output'])

In [16]:
# with open('hehe.txt', 'w') as f:
#     f.write(full)

In [17]:
# ans

In [18]:
# h = set([ str(_).split(',')[0] for _ in ans.values()])
# h

In [16]:
def generate_analysis_report(data,output_path,verbose,level):
    report_df = pd.DataFrame() # Dataframe for analysis
    # cm=np.zeros((4,4)) # Confusion Matrix
    uids=list(data.keys())
    c = 0
    for uid in tqdm(uids, desc="Processing"):
        best_report_df=process_uid_(uid,data,level=level)
        if best_report_df is not None:
            report_df=pd.concat([report_df,best_report_df])
        else:
            c+= 1
            # print(uid)
    csv_path=os.path.join(output_path,f"report_df_{level}.csv")

    try:
        report_df.to_csv(csv_path, index=None)
        # logger.success(f"Report is saved at {csv_path}")
    except Exception as e:
        # logger.error(f"Error saving csv file at {csv_path}")
        # if verbose:
        #    print(traceback.print_exc())
        print('et_o_et')
    print(c)
    # if verbose:
        # logger.info("Calculating Metrics...")

    eval_dict = {}

    line_metrics = report_df[(report_df['line_total_gt'] > 0)][['line_recall', 'line_precision', 'line_f1']].mean() * 100
    eval_dict['line'] = {
        'recall': line_metrics['line_recall'],
        'precision': line_metrics['line_precision'],
        'f1': line_metrics['line_f1']
    }

    # Mean Recall, Precision, F1 for Arc
    arc_metrics = report_df[(report_df['arc_total_gt'] > 0)][['arc_recall', 'arc_precision', 'arc_f1']].mean() * 100
    eval_dict['arc'] = {
        'recall': arc_metrics['arc_recall'],
        'precision': arc_metrics['arc_precision'],
        'f1': arc_metrics['arc_f1']
    }

    # Mean Recall, Precision, F1 for Circle
    circle_metrics = report_df[(report_df['circle_total_gt'] > 0)][['circle_recall', 'circle_precision', 'circle_f1']].mean() * 100
    eval_dict['circle'] = {
        'recall': circle_metrics['circle_recall'],
        'precision': circle_metrics['circle_precision'],
        'f1': circle_metrics['circle_f1']
    }

    # Mean Recall, Precision, F1 for Extrusion
    ext_recall = report_df['num_ext'] / report_df['num_ext_gt']
    ext_precision = report_df['num_ext'] / report_df['num_ext_pred']
    ext_f1 = 2 * ext_recall * ext_precision / (ext_recall + ext_precision)
    extrusion_metrics = {
        'recall': ext_recall.mean() * 100,
        'precision': ext_precision.mean() * 100,
        'f1': ext_f1.mean() * 100
    }
    eval_dict.update({'extrusion': extrusion_metrics})

    
    # Update Chamfer Distance
    eval_dict['cd']={}
    eval_dict['cd']['median']=report_df['cd'][report_df['cd']>0].median()
    eval_dict['cd']['mean']=report_df['cd'][report_df['cd']>0].mean()
    # eval_dict['invalidity_ratio_percentage']=report_df['cd'][report_df['cd']<0].count()*100/len(report_df)
    eval_dict['invalidity_ratio_percentage'] = c*100/(c + len(report_df))
    
    if verbose:
        json_formatted_str = json.dumps(eval_dict, indent=4)
        print(json_formatted_str)

    mean_report_path=os.path.join(output_path,f"mean_report_{level}.json")

    with open(mean_report_path,"w") as f:
        json.dump(eval_dict,f, indent=4)



def process_vec(pred_vec,gt_vec,bit,uid):
    try:
        # pred_cad=CADSequence.from_vec(pred_vec,2,8,denumericalize=False)
        # gt_cad=CADSequence.from_vec(gt_vec,2,8,denumericalize=False)

        report_df,cm=gt_vec.generate_report(pred_vec,uid)
        return report_df,cm
    except Exception as e:
        #print(e)
        return None,None

def process_uid_(uid,data,level):
    try:
        gt_vec = data[uid][level]['gt_cad_vec']
        all_cd = data[uid][level]['cd']
        best_index = 0
        pred_vec = data[uid][level]['pred_cad_vec'][best_index]
        df, _ = process_vec(pred_vec, gt_vec, 8, uid)
        df['cd'] = all_cd[best_index]
    
        return df

    except Exception as e:
        return None

In [17]:
output_dir=create_path_with_time('output')
output_path=os.path.join(output_dir,'final_')
ensure_dir(output_path)
generate_analysis_report(data=output_data,output_path=output_path,
                                 verbose = True, level=str(level))

Processing: 100%|██████████████████████████████████████████████████████████████████| 7739/7739 [05:25<00:00, 23.80it/s]


53
{
    "line": {
        "recall": 97.47185627851643,
        "precision": 98.70134649813748,
        "f1": 97.7649752088584
    },
    "arc": {
        "recall": 92.52729794025556,
        "precision": 94.33561916494313,
        "f1": 92.82269520886435
    },
    "circle": {
        "recall": 96.66938802538617,
        "precision": 97.38351054020345,
        "f1": 96.62449869259096
    },
    "extrusion": {
        "recall": 99.43841926570263,
        "precision": 99.87072971265009,
        "f1": 99.55408311736088
    },
    "cd": {
        "median": 0.15279586143174673,
        "mean": 3.7622644656349986
    },
    "invalidity_ratio_percentage": 0.6848430029719602
}


## TESTING

In [12]:
# e = '''
# {"final_shape": "", "parts": {"part_1": {"coordinate_system": {"Euler Angles": [0.0, 0.0, 0.0], "Translation Vector": [0.0, 0.0, 0.0]}, "sketch": {"face_1": {"loop_1": {"line_1": {"Start Point": [0.0, 0.0], "End Point": [0.5806, 0.0]}, "line_2": {"Start Point": [0.5806, 0.0], "End Point": [0.5806, 0.3629]}, "line_3": {"Start Point": [0.5806, 0.3629], "End Point": [0.0, 0.3629]}, "line_4": {"Start Point": [0.0, 0.3629], "End Point": [0.0, 0.0]}}, "loop_2": {"circle_1": {"Center": [0.2903, 0.18140000000000002], "Radius": 0.1667}}}}, "extrusion": {"extrude_depth_towards_normal": 0.75, "extrude_depth_opposite_normal": 0.0, "sketch_scale": 0.5806, "operation": "NewBodyFeatureOperation"}, "description": {"shape": "", "length": 0.5806130922, "width": 0.5806130922, "height": 0.75}}}}
# '''
# # for i in data:
# #     if i['uid'] == '0028/00289243':
# #         e = i['predicted_output']
# #         break

# cad_err = CADSequence.from_minimal_json(json.loads(e))
# cad_err._json()

In [13]:
# e = '''
# {"final_shape": "", "parts": {"part_1": {"coordinate_system": {"Euler Angles": [0.0, 0.0, 0.0], "Translation Vector": [0.0, 0.0, 0.0]}, "sketch": {"face_1": {"loop_1": {"arc_1": {"Start Point": [0.0, 0.025], "Mid Point": [0.007300000000000001, 0.007300000000000001], "End Point": [0.025, 0.0]}, "line_1": {"Start Point": [0.025, 0.0], "End Point": [0.725, 0.0]}, "arc_2": {"Start Point": [0.725, 0.0], "Mid Point": [0.725, 0.0484], "End Point": [0.75, 0.0484]}, "line_2": {"Start Point": [0.75, 0.0484], "End Point": [0.75, 0.45]}, "arc_3": {"Start Point": [0.75, 0.45], "Mid Point": [0.725, 0.45], "End Point": [0.725, 0.5]}, "line_3": {"Start Point": [0.725, 0.5], "End Point": [0.075, 0.5]}, "arc_4": {"Start Point": [0.075, 0.5], "Mid Point": [0.007300000000000001, 0.5007], "End Point": [0.0, 0.5007]}, "line_4": {"Start Point": [0.0, 0.5007], "End Point": [0.0, 0.025]}}}, "face_2": {"loop_1": {"arc_1": {"Start Point": [0.0, 0.025], "Mid Point": [0.007300000000000001, 0.007300000000000001], "End Point": [0.025, 0.0]}, "line_1": {"Start Point": [0.025, 0.0], "End Point": [0.725, 0.0]}, "arc_2": {"Start Point": [0.725, 0.0], "Mid Point": [0.725, 0.0484], "End Point": [0.75, 0.0484]}, "line_2": {"Start Point": [0.75, 0.0484], "End Point": [0.75, 0.45]}, "arc_3": {"Start Point": [0.75, 0.45], "Mid Point": [0.725, 0.45], "End Point": [0.725, 0.5]}, "line_3": {"Start Point": [0.725, 0.5], "End Point": [0.075, 0.5]}, "arc_4": {"Start Point": [0.075, 0.5], "Mid Point": [0.007300000000000001, 0.5007], "End Point": [0.0, 0.5007]}, "line_4": {"Start Point": [0.0, 0.5007], "End Point": [0.0, 0.025]}}}, "face_3": {"loop_1": {"arc_1": {"Start Point": [0.075, 0.05], "Mid Point": [0.007300000000000001, 0.007300000000000001], "End Point": [0.025, 0.05]}, "line_1": {"Start Point": [0.025, 0.05], "End Point": [0.725, 0.05]}, "arc_2": {"Start Point": [0.725, 0.05], "Mid Point": [0.725, 0.0484], "End Point": [0.75, 0.0484]}, "line_2": {"Start Point": [0.75, 0.0484], "End Point": [0.75, 0.45]}, "arc_3": {"Start Point": [0.75, 0.45], "Mid Point": [0.725, 0.45], "End Point": [0.725, 0.5]}, "line_3": {"Start Point": [0.725, 0.5], "End Point": [0.075, 0.5]}, "arc_4": {"Start Point": [0.075, 0.5], "Mid Point": [0.007300000000000001, 0.5007], "End Point": [0.0, 0.5007]}, "line_4": {"Start Point": [0.0, 0.5007], "End Point": [0.0, 0.025]}}}}, "extrusion": {"extrude_depth_towards_normal": 0.0005, "extrude_depth_opposite_normal": 0.0, "sketch_scale": 0.75, "operation": "NewBodyFeatureOperation"}, "description": {"shape": "", "length": 0.75, "width": 0.75, "height": 0.0005}}}}
# '''
# # cad_err = CADSequence.from_minimal_json(json.loads(e))
# # cad_err.sample_points()

In [14]:
# output_data['0028/00289243']['expert']['pred_cad_vec'][0]._json()

In [15]:
from collections import defaultdict

x = 0
err_xx = defaultdict(list)

for sample in error_samples:
    try:
        json_cad = json.loads(sample['predicted_output'])
        cad = ff(fix_cad_json(clean_json(json_cad)))
        cad = CADSequence.from_minimal_json(cad)
        cad.sample_points()
    except Exception as e:
        x += 1
        error_type = f"{type(e).__name__}: {str(e)}"
        err_xx[error_type].append(sample['uid'])
        print(error_type, ' ---- ', sample['uid'])

# print(x)
# print(len(error_samples))

JSONDecodeError: Expecting ',' delimiter: line 1 column 11507 (char 11506)  ----  0035/00353920
JSONDecodeError: Expecting ',' delimiter: line 1 column 11524 (char 11523)  ----  0077/00778682
JSONDecodeError: Expecting ',' delimiter: line 1 column 12280 (char 12279)  ----  0077/00774679
Exception: Invalid Model  ----  0077/00779074
JSONDecodeError: Expecting ',' delimiter: line 1 column 4862 (char 4861)  ----  0069/00697339
JSONDecodeError: Expecting ',' delimiter: line 1 column 3376 (char 3375)  ----  0069/00697032
JSONDecodeError: Expecting ',' delimiter: line 1 column 1137 (char 1136)  ----  0098/00981569
JSONDecodeError: Expecting ',' delimiter: line 1 column 14174 (char 14173)  ----  0098/00985194
JSONDecodeError: Extra data: line 1 column 4167 (char 4166)  ----  0098/00987045
Exception: Invalid Model  ----  0098/00983119
JSONDecodeError: Extra data: line 1 column 3302 (char 3301)  ----  0066/00662647
Exception: Invalid Model  ----  0066/00669966
JSONDecodeError: Extra data: line 

In [16]:
print(x, ' ', len(error_samples))

310   318


In [17]:
# # y = data['0025/00254881']['expert']['pred_cad_vec'][0]._json()
# # y
for j in data:
    if j['uid'] =='0035/00350196':
        err = j['predicted_output']
        jxxx = j
        print('??')
        break

??


In [18]:
cad_err.points

NameError: name 'cad_err' is not defined

In [ ]:
cad_err = CADSequence.from_minimal_json(clean_json(json.loads(err)))
cad_err.sample_points()
gt = CADSequence.from_minimal_json(json.loads(jxxx['ground_truth_output']))
gt.sample_points()

cd = (
    chamfer_dist(
        normalize_pc(gt_cad.points),
        normalize_pc(pred_cad.points),
    )
    * 1000
)
print(cd)

In [ ]:
# cad_err = fix_json_cad(json.loads(err))
cad_err = json.loads(err)
# cad_err = err
cad_err = CADSequence.from_minimal_json(cad_err)
cad_err.sample_points()

gt = CADSequence.from_minimal_json(json.loads(jxxx['ground_truth_output']))
gt.sample_points()

cd = (
    chamfer_dist(
        normalize_pc(gt_cad.points),
        normalize_pc(pred_cad.points),
    )
    * 1000
)
print(cd)

In [ ]:
# cleaned_data = fix_cad_json(json.loads(e))
# cleaned_data

In [ ]:
# cad_err = CADSequence.from_minimal_json(cleaned_data)
# c = cad_err.sample_points()

In [2]:
dataset = load_dataset("TruongSinhAI/DEEPCAD-Text2Json-EnVi")

In [3]:
dataset

DatasetDict({
    train_en: Dataset({
        features: ['uid', 'input', 'output', 'prompt'],
        num_rows: 155783
    })
    train_vi: Dataset({
        features: ['uid', 'input', 'output', 'prompt'],
        num_rows: 155783
    })
    val_en: Dataset({
        features: ['uid', 'input', 'output', 'prompt'],
        num_rows: 8814
    })
    val_vi: Dataset({
        features: ['uid', 'input', 'output', 'prompt'],
        num_rows: 8814
    })
    test_en: Dataset({
        features: ['uid', 'input', 'output', 'prompt'],
        num_rows: 7924
    })
    test_vi: Dataset({
        features: ['uid', 'input', 'output', 'prompt'],
        num_rows: 7924
    })
    subset1000_en: Dataset({
        features: ['uid', 'input', 'output', 'prompt'],
        num_rows: 1000
    })
    subset1000_vi: Dataset({
        features: ['uid', 'input', 'output', 'prompt'],
        num_rows: 1000
    })
})

In [4]:
from tqdm import tqdm
errors = {}
for split in ['train_en', 'val_en', 'test_en']:
    errors[split] = []
    for sample in tqdm(dataset[split]):
        uid = sample['uid']
        output = sample['output']
        try:
            g = CADSequence.from_minimal_json(json.loads(output.replace("'", '"'))).create_cad_model()
        except Exception as e:
            errors[split].append(uid)


100%|██████████████████████████████████████████████████████████████████████████████| 7924/7924 [03:40<00:00, 35.94it/s]


In [6]:
for split in errors:
    print(split, ' : ', len(errors[split]))

train_en  :  373
val_en  :  14
test_en  :  14


In [8]:
x = set()
for split in ['train_en', 'val_en', 'test_en']:
    for sample in tqdm(dataset[split]):
        uid = sample['uid']
        if uid in errors[split]:
            output = sample['output']
            try:
                g = CADSequence.from_minimal_json(json.loads(output.replace("'", '"'))).create_cad_model()
            except Exception as e:
                x.add(e)

100%|███████████████████████████████████████████████████████████████████████████| 7924/7924 [00:00<00:00, 14282.08it/s]


In [7]:
with open("errors.json", "w", encoding="utf-8") as f:
    json.dump(errors, f, ensure_ascii=False, indent=4)